In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
var DB = OpenOrCreateDatabase(@"W:\work\scratch\jw52xeqa\DB_rotSphereBenchmark"); DB.Sessions

Opening existing database 'W:\work\scratch\jw52xeqa\DB_rotSphereBenchmark'.


#0: rotCube+Inlet	J17496_k3_t60_NSE__c64_mue_0.01*	09/16/2021 22:36:18	ce253fdb...
#1: rotCube+Inlet	J17496_k3_t60_NSE__c64_mue_0.1*	09/16/2021 22:33:35	cfd627b8...
#2: rotCube+Inlet	J10125_k4_t60_NSE__c64_mue_0.01__c64_mue_0.01*	09/16/2021 22:24:51	35d08e74...
#3: rotCube+Inlet	J10125_k4_t60_NSE__c64_mue_0.1__c64_mue_0.1*	09/16/2021 22:11:09	ae180d73...
#4: rotCube+Inlet	J10125_k4_t60_NSE__c64_mue_0.1*	09/16/2021 22:07:23	879e8fe0...
#5: rotCube+Inlet	J12288_k3_t60_NSE__c64_mue_0.01*	09/16/2021 21:34:00	2868a46e...
#6: rotCube+Inlet	J10125_k3_t60_NSE_c128_mue_0.1__c128_mue_0.1*	09/16/2021 01:59:20	7d880f3c...
#7: test	J17496_k4_t1_Stokes__c64_Re50_mue_1_c64_Re50_mue_1	09/16/2021 00:00:54	44e8924d...
#8: test	J31944_k3_t1_Stokes__c64_Re50_mue_1_c64_Re50_mue_1	09/16/2021 00:00:39	b7e488f5...
#9: test	J65856_k2_t1_Stokes__c64_Re50_mue_1_c64_Re50_mue_1	09/16/2021 00:00:26	1289adb3...
#10: rotCube+Inlet	J10125_k3_t60_NSE_c64_mue_0.001*	09/15/2021 15:29:43	02b21330...
#11: rotCube+Inlet_tes

In [ ]:
string WFlowName = "Benchmark_4";
var wfmgr = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm;
wfmgr.Init(WFlowName); wfmgr.Sessions

Project name is set to 'Benchmark_4'.


#0: Benchmark_4	J12288_k4_t1_Stokes_c256_Re50_mue_1	09/15/2021 00:41:05	db96d01a...
#1: Benchmark_4	J12288_k4_t1_Stokes_c128_Re50_mue_1	09/15/2021 00:40:02	63a068d5...
#2: Benchmark_4	J12288_k4_t1_Stokes_c8_Re50_mue_1	09/15/2021 00:33:54	7c7cda83...
#3: Benchmark_4	J12288_k4_t1_Stokes_c32_Re50_mue_1	09/15/2021 00:39:34	f7b9d04b...
#4: Benchmark_4	J12288_k4_t1_Stokes_c16_Re50_mue_1	09/15/2021 00:33:57	bc819739...
#5: Benchmark_4	J46875_k2_t1_Stokes_c8_Re50_mue_1	09/15/2021 00:31:23	143ea4c2...
#6: Benchmark_4	J46875_k2_t1_Stokes_c16_Re50_mue_1	09/15/2021 00:31:55	a0a2c07a...
#7: Benchmark_4	J46875_k2_t1_Stokes_c32_Re50_mue_1	09/15/2021 00:31:54	6b249cab...
#8: Benchmark_4	J12288_k4_t1_Stokes_c64_Re50_mue_1*	09/15/2021 00:39:59	3122c3a3...
#9: Benchmark_4	J46875_k2_t1_Stokes_c256_Re50_mue_1	09/15/2021 00:32:31	3a8d9099...
#10: Benchmark_4	J46875_k2_t1_Stokes_c64_Re50_mue_1	09/15/2021 00:32:23	47dcdbab...
#11: Benchmark_4	J46875_k2_t1_Stokes_c128_Re50_mue_1	09/15/2021 00:32:28	86c6770d...

In [ ]:
/*
CG projection               -> ProjectDGField                               BoSSS.Foundation.ConstrainedDGprojection.ConstrainedDGField_Global.ProjectDGField
dyn Loadbalancing           -> DoLoadbalancing                              BoSSS.Solution.Application`1.DoLoadbalancing
AMR                         -> DoMeshAdaption                               BoSSS.Solution.Application`1.DoMeshAdaption
compile Cut-cell quadrature -> Rule_Compilation_SayeGaussRuleFactory        Rule_Compilation_SayeGaussRuleFactory
compile standard quadrature -> Rule_Compilation_StandardQuadRuleFactory     Rule_Compilation_StandardQuadRuleFactory
matrix assembly             -> XdgMatrixAssembly                            BoSSS.Solution.XdgTimestepping.XdgBDFTimestepping.AssembleMatrixCallback
agg basis init              -> Aggregation_basis_init_time                  BoSSS.Solution.AdvancedSolvers.XdgAggregationBasis.Update
solver init                 -> Solver_Init_time                             Slv Init
solver run                  -> Solver_Run_time                              Slv Iter
I/O                         -> SaveToDatabase                               BoSSS.Solution.Application`1.SaveToDatabase
*/

In [ ]:
static var PreComputeProfiling = new Dictionary<System.Guid,MethodCallRecord[]>();
foreach(var sess in wfmgr.Sessions){
    MethodCallRecord[] mcrs = null;
    try{
        mcrs = sess.GetProfiling();
    } catch (Exception ex){
        Console.WriteLine(ex.Message);
    }
    PreComputeProfiling.Add(sess.ID, mcrs);
}

Unable to find profiling information.


In [ ]:
static class AddCols {
    static public MethodCallRecord[] GetMCR(ISessionInfo SI){
        MethodCallRecord[] mcrs = null;
        PreComputeProfiling.TryGetValue(SI.ID,out mcrs);
        return mcrs;
    }
    
    static private double TimeOnProc(MethodCallRecord mcr, string method){
        double time = 0;
        foreach(var child in mcr.FindChildren(method)){
            time += child.TimeSpentInMethod.TotalSeconds;
            //time += child.TimeSpentInMPIBlocking.TotalSeconds;
        }
        return time;
    }
    
    static private double AverageTime(ISessionInfo SI,string method){
        var mcrs       = GetMCR(SI);
        double avtime = 0;

        foreach(var mcr in mcrs){
            avtime+=TimeOnProc(mcr,method);
        }
        avtime = avtime / mcrs.Count();
        return avtime;
    }

    static private double MaxTime(ISessionInfo SI,string method){
        var mcrs       = GetMCR(SI);
        double maxtime = -1;

        foreach(var mcr in mcrs){
            double time =TimeOnProc(mcr,method);    
            maxtime = time>maxtime?time:maxtime;
        }
        return maxtime;
    }

    static public object LSolverIter(ISessionInfo SI){
        var mcr          = GetMCR(SI)[0];
        string searchstr = "BoSSS.Solution.AdvancedSolvers.DirectSolver.Solve";
        var ndS = mcr.FindChildren(searchstr);
        var nd  = ndS.ElementAt(0);
        return nd.CallCount;
    }

    static public object NLSolverIter(ISessionInfo SI){
        var mcr          = GetMCR(SI)[0];
        string searchstr = "BoSSS.Solution.AdvancedSolvers.Newton.GlobalizedNewton";
        var ndS = mcr.FindChildren(searchstr);
        var nd  = ndS.ElementAt(0);
        return nd.CallCount;
    }

    static public object TotalRuntime(ISessionInfo SI){
        return AverageTime(SI,"root_frame");
    }
    static public object MatrixAssembly_time(ISessionInfo SI) {
        return AverageTime(SI,"BoSSS.Solution.XdgTimestepping.XdgBDFTimestepping.AssembleMatrixCallback");
    }
    static public object AggregationBaseInit_time(ISessionInfo SI) {
        return AverageTime(SI,"BoSSS.Solution.AdvancedSolvers.XdgAggregationBasis.Update");
    }
    static public object SolverInit_time(ISessionInfo SI) {
        return AverageTime(SI,"Slv Init");
    }
    static public object SolverRun_time(ISessionInfo SI) {
        return AverageTime(SI,"Slv Iter");
    }
    static public object DataIO_time(ISessionInfo SI){
        double sum = 0;
        sum+=AverageTime(SI,"BoSSS.Solution.Application`1.CreateOrLoadGrid");
        sum+=AverageTime(SI,"BoSSS.Solution.Application`1.SaveToDatabase");
        return sum;
    }
    static public object CGProjection_time(ISessionInfo SI){
        return AverageTime(SI,"BoSSS.Foundation.ConstrainedDGprojection.ConstrainedDGField_Global.ProjectDGField");
    }
    static public object SayeCompile_time(ISessionInfo SI){
        return AverageTime(SI,"Rule_Compilation_SayeGaussRuleFactory");
    }
    static public object StandardCompile_time(ISessionInfo SI){
        return AverageTime(SI,"Rule_Compilation_StandardQuadRuleFactory");
    }
    static public object AMR_time(ISessionInfo SI){
        return AverageTime(SI,"BoSSS.Solution.Application`1.DoMeshAdaption");
    }
    static public object LoadBal_time(ISessionInfo SI){
        return AverageTime(SI,"BoSSS.Solution.Application`1.DoLoadbalancing");
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.ComputeNodeNames.Count();
    }
    static public object NoOfNodes(ISessionInfo SI){
        return SI.ComputeNodeNames.GroupBy(s=>s).Count();
    }
    static public object MGDepth(ISessionInfo SI){
        var mcr = GetMCR(SI)[0];
        double NInit  = mcr.FindChildren("BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Init").ToArray().Length;
        return NInit + 1;
    }
}

In [ ]:
var SI = wfmgr.Sessions.Pick(1);
AddCols.MGDepth(SI);

In [ ]:
wfmgr.AdditionalSessionTableColums.Clear();
wfmgr.AdditionalSessionTableColums.Add("LSolverIter", AddCols.LSolverIter);
wfmgr.AdditionalSessionTableColums.Add("NLSolverIter", AddCols.NLSolverIter);
wfmgr.AdditionalSessionTableColums.Add("TotalRuntime", AddCols.TotalRuntime);
wfmgr.AdditionalSessionTableColums.Add("MatrixAssembly_time", AddCols.MatrixAssembly_time);
wfmgr.AdditionalSessionTableColums.Add("AggregationBaseInit_time", AddCols.AggregationBaseInit_time);
wfmgr.AdditionalSessionTableColums.Add("DataIO_time", AddCols.DataIO_time);
wfmgr.AdditionalSessionTableColums.Add("CGProjection_time", AddCols.CGProjection_time);
wfmgr.AdditionalSessionTableColums.Add("SayeCompile_time", AddCols.SayeCompile_time);
wfmgr.AdditionalSessionTableColums.Add("StandardCompile_time",AddCols.StandardCompile_time);
wfmgr.AdditionalSessionTableColums.Add("AMR_time",AddCols.AMR_time);
wfmgr.AdditionalSessionTableColums.Add("LoadBal_time",AddCols.LoadBal_time);
wfmgr.AdditionalSessionTableColums.Add("NoOfCores",AddCols.NoOfCores);
wfmgr.AdditionalSessionTableColums.Add("NoOfNodes",AddCols.NoOfNodes);
wfmgr.AdditionalSessionTableColums.Add("SolverInit_time", AddCols.SolverInit_time);
wfmgr.AdditionalSessionTableColums.Add("SolverRun_time", AddCols.SolverRun_time);
wfmgr.AdditionalSessionTableColums.Add("MGDepth",AddCols.MGDepth);

In [ ]:
var SessTab = wfmgr.SessionTable;
SessTab.GetColumnNames().OrderBy(s=>s)

In [ ]:
// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = SessTab.GetColumnNames().ToList();
AllCols.Remove("Session");

In [ ]:
SessTab.GetColumnNames().Where(s=>s.Contains("Depth"))

In [ ]:
SessTab = SessTab.ExtractColumns(AllCols.ToArray());
// Filename
string thisdir = Directory.GetCurrentDirectory();
DirectoryInfo dir = Directory.CreateDirectory(thisdir+@"\"+WFlowName);
string dirpath    = dir.FullName;
var now           = DateTime.Now;
SessTab.TableName = WFlowName + "_" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath    = Path.Combine(dirpath, SessTab.TableName + ".json");
SessTab.SaveToFile(docpath); docpath

In [ ]:
var SubTab = SessTab.ExtractColumns("LSolverIter", "NLSolverIter", "TotalRuntime", "MatrixAssembly_time", "AggregationBaseInit_time", "DataIO_time", "CGProjection_time",
"SayeCompile_time", "StandardCompile_time","AMR_time","LoadBal_time","NoOfCores","NoOfNodes","SolverInit_time", "SolverRun_time", "MGDepth","DGdegree:Velocity*"); SubTab.Print()

In [ ]:
SubTab.ToCSVFile(dirpath+@"\table.csv");

In [ ]:
docpath